In [37]:
import pandas as pd

df = pd.read_csv("../final_data/data_260116.csv")

pd.set_option("display.max_columns", None)
df.head()

,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day
0,2025-07-28,不發,控制組,14天在其他尖峰預估車資,中區,333,91,45,74,17,34,11,0.273273,0.135135,0.222222,0.051051,0.102102,0.033033,0.523529,0.749091,20,114,0,2,9,0,24,0,0.897959,0.000000,0.500000,7
1,2025-07-28,不發,控制組,14天在其他尖峰預估車資,北區,1220,228,161,165,63,129,32,0.186885,0.131967,0.135246,0.051639,0.105738,0.026230,0.793101,0.779062,107,583,0,12,299,0,30,1,0.732143,0.000000,0.000000,7
2,2025-07-28,不發,控制組,14天在其他尖峰預估車資,南區,374,73,51,52,21,36,15,0.195187,0.136364,0.139037,0.056150,0.096257,0.040107,0.751944,0.850000,30,129,1,2,10,1,21,0,0.809524,0.366667,0.583333,7
3,2025-07-28,不發,控制組,14天在晚尖峰預估車資,中區,106,34,23,23,11,15,8,0.320755,0.216981,0.216981,0.103774,0.141509,0.075472,0.744000,0.917500,10,36,0,2,1,0,0,0,0.897959,0.000000,0.500000,7
4,2025-07-28,不發,控制組,14天在晚尖峰預估車資,北區,288,56,42,48,8,37,5,0.194444,0.145833,0.166667,0.027778,0.128472,0.017361,0.845676,0.750000,37,90,0,2,56,0,6,0,0.732143,0.000000,0.000000,7


In [38]:
import numpy as np
import pandas as pd


INPUT_PATH = "../final_data/data_260116.csv"
OUTPUT_PATH = "../final_data/data_260119.csv"


HOLIDAY_COL = "has_national_holiday"


def _detect_city_col(cols):
    if "city_group" in cols:
        return "city_group"
    if "city" in cols:
        return "city"
    raise ValueError("Cannot find city column. Expected 'city_group' or 'city'.")


def _detect_holiday_col(cols):
    if HOLIDAY_COL is not None and HOLIDAY_COL in cols:
        return HOLIDAY_COL

    candidates = [
        "holiday_week", "is_holiday_week", "has_holiday_week", "week_has_holiday",
        "is_national_holiday_week", "has_national_holiday", "holiday_flag"
    ]
    for c in candidates:
        if c in cols:
            return c
    return None


def build_features_keep_used_only(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # --- Date parsing ---
    df["experiment_date"] = pd.to_datetime(df["experiment_date"])

    # =========================
    # (補上) 移除 2026-01-12
    # =========================
    df = df[df["experiment_date"] != pd.Timestamp("2026-01-12")].copy()

    # =========================
    # (補上) 國定假日週旗標 has_national_holiday
    # =========================
    holidays = pd.to_datetime([
        "2025-09-29",
        "2025-10-06",
        "2025-10-10",
        "2025-10-24",
        "2025-12-25",
        "2026-01-01",
    ])

    week_start = df["experiment_date"].dt.normalize()
    week_end = week_start + pd.Timedelta(days=6)

    has_holiday = np.logical_or.reduce([
        (h >= week_start) & (h <= week_end)
        for h in holidays
    ])
    df["has_national_holiday"] = has_holiday.astype(int)

    # --- Identify key columns ---
    city_col = _detect_city_col(df.columns)
    holiday_col = _detect_holiday_col(df.columns)

    gkey = ["treatment", "source", "ops_type_merged", city_col]
    df = df.sort_values(gkey + ["experiment_date"])

    # --- log1p(user_cnt) ---
    if "user_cnt" not in df.columns:
        raise ValueError("Missing required column: user_cnt")
    df["log1p_user_cnt"] = np.log1p(df["user_cnt"].astype(float))

    # =========================
    # Coupon processing
    # =========================
    coupon_cols_all = [
        "coupon_BD_total", "coupon_CDP_total", "coupon_folk_total",
        "coupon_growth_other_total", "coupon_MGM_total", "coupon_MKT_total",
        "coupon_register_total", "coupon_daily_total",
    ]
    coupon_cols = [c for c in coupon_cols_all if c in df.columns]

    # If none exist, we skip coupon features gracefully
    denom = df["user_cnt"].clip(lower=1).astype(float)

    # coupon_total = sum of all available coupon_*_total
    if coupon_cols:
        df[coupon_cols] = df[coupon_cols].fillna(0)
        df["coupon_total"] = df[coupon_cols].sum(axis=1)

        # per_user then log1p
        coupon_log_per_user_cols = []
        for c in coupon_cols + ["coupon_total"]:
            per_user = df[c].astype(float) / denom
            log_col = f"log_{c}_per_user"
            df[log_col] = np.log1p(per_user)
            coupon_log_per_user_cols.append(log_col)
    else:
        df["coupon_total"] = 0.0
        coupon_log_per_user_cols = []

    # =========================
    # Base lag/roll features
    # =========================
    required_targets = ["trip_cnt_per_user", "nonrepeat_cnt_per_user"]
    for t in required_targets:
        if t not in df.columns:
            raise ValueError(f"Missing required target column: {t}")

    base_lag_cols = [
        "trip_cnt_per_user",
        "nonrepeat_cnt_per_user",
        "weekday_nonrepeat_cnt_per_user", "weekend_nonrepeat_cnt_per_user",
        "weekday_trip_cnt_per_user", "weekend_trip_cnt_per_user",
        "weekday_match_rate", "weekend_match_rate",
    ]
    base_lag_cols = [c for c in base_lag_cols if c in df.columns]

    created_feature_cols = []

    def add_lag_roll(col_name: str):
        s = df.groupby(gkey, sort=False)[col_name]

        lag1 = f"{col_name}_lag1"
        lag2 = f"{col_name}_lag2"
        roll4 = f"{col_name}_roll4"

        df[lag1] = s.shift(1)
        df[lag2] = s.shift(2)
        df[roll4] = s.shift(1).rolling(4, min_periods=2).mean()

        for feat in (lag1, lag2, roll4):
            flag = f"{feat}_isna"
            df[flag] = df[feat].isna().astype(int)
            df[feat] = df[feat].fillna(0)
            created_feature_cols.extend([feat, flag])

    for col in base_lag_cols:
        add_lag_roll(col)

    for col in coupon_log_per_user_cols:
        add_lag_roll(col)

    # =========================
    # Delta features (from lagged targets)
    # =========================
    df["delta_trip_per_user"] = df["trip_cnt_per_user_lag1"] - df["trip_cnt_per_user_lag2"]
    df["delta_nonrepeat_per_user"] = df["nonrepeat_cnt_per_user_lag1"] - df["nonrepeat_cnt_per_user_lag2"]
    created_feature_cols.extend(["delta_trip_per_user", "delta_nonrepeat_per_user"])

    # =========================
    # Keep ONLY columns used for modeling
    # =========================
    id_cols = ["experiment_date", "treatment", "source", "ops_type_merged", city_col]
    target_cols = ["trip_cnt_per_user", "nonrepeat_cnt_per_user"]

    known_now_cols = ["log1p_user_cnt"]
    for c in ["avg_rainy_day", "avg_rainy_weekday", "avg_rainy_weekend", "mgm_day"]:
        if c in df.columns:
            known_now_cols.append(c)
    if holiday_col is not None:
        known_now_cols.append(holiday_col)

    keep_cols = id_cols + target_cols + known_now_cols + created_feature_cols

    seen = set()
    keep_cols = [c for c in keep_cols if (c in df.columns and (c not in seen and not seen.add(c)))]

    df_model = df[keep_cols].copy()
    return df_model


if __name__ == "__main__":
    df = pd.read_csv(INPUT_PATH)
    df_model = build_features_keep_used_only(df)

    df_model.to_csv(OUTPUT_PATH, index=False)

    print("Saved model-ready data to:", OUTPUT_PATH)
    print("Shape:", df_model.shape)
    print("Columns:", len(df_model.columns))


C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_22324\1403022800.py:139: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[flag] = df[feat].isna().astype(int)
C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_22324\1403022800.py:139: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[flag] = df[feat].isna().astype(int)
C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_22324\1403022800.py:139: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

Saved model-ready data to: ../final_data/data_260119.csv
Shape: (2856, 117)
Columns: 117


In [39]:
import pandas as pd

df = pd.read_csv("../final_data/data_260119.csv")

pd.set_option("display.max_columns", None)
df.head()

,experiment_date,treatment,source,ops_type_merged,city_group,trip_cnt_per_user,nonrepeat_cnt_per_user,log1p_user_cnt,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day,has_national_holiday,trip_cnt_per_user_lag1,trip_cnt_per_user_lag1_isna,trip_cnt_per_user_lag2,trip_cnt_per_user_lag2_isna,trip_cnt_per_user_roll4,trip_cnt_per_user_roll4_isna,nonrepeat_cnt_per_user_lag1,nonrepeat_cnt_per_user_lag1_isna,nonrepeat_cnt_per_user_lag2,nonrepeat_cnt_per_user_lag2_isna,nonrepeat_cnt_per_user_roll4,nonrepeat_cnt_per_user_roll4_isna,weekday_nonrepeat_cnt_per_user_lag1,weekday_nonrepeat_cnt_per_user_lag1_isna,weekday_nonrepeat_cnt_per_user_lag2,weekday_nonrepeat_cnt_per_user_lag2_isna,weekday_nonrepeat_cnt_per_user_roll4,weekday_nonrepeat_cnt_per_user_roll4_isna,weekend_nonrepeat_cnt_per_user_lag1,weekend_nonrepeat_cnt_per_user_lag1_isna,weekend_nonrepeat_cnt_per_user_lag2,weekend_nonrepeat_cnt_per_user_lag2_isna,weekend_nonrepeat_cnt_per_user_roll4,weekend_nonrepeat_cnt_per_user_roll4_isna,weekday_trip_cnt_per_user_lag1,weekday_trip_cnt_per_user_lag1_isna,weekday_trip_cnt_per_user_lag2,weekday_trip_cnt_per_user_lag2_isna,weekday_trip_cnt_per_user_roll4,weekday_trip_cnt_per_user_roll4_isna,weekend_trip_cnt_per_user_lag1,weekend_trip_cnt_per_user_lag1_isna,weekend_trip_cnt_per_user_lag2,weekend_trip_cnt_per_user_lag2_isna,weekend_trip_cnt_per_user_roll4,weekend_trip_cnt_per_user_roll4_isna,weekday_match_rate_lag1,weekday_match_rate_lag1_isna,weekday_match_rate_lag2,weekday_match_rate_lag2_isna,weekday_match_rate_roll4,weekday_match_rate_roll4_isna,weekend_match_rate_lag1,weekend_match_rate_lag1_isna,weekend_match_rate_lag2,weekend_match_rate_lag2_isna,weekend_match_rate_roll4,weekend_match_rate_roll4_isna,log_coupon_BD_total_per_user_lag1,log_coupon_BD_total_per_user_lag1_isna,log_coupon_BD_total_per_user_lag2,log_coupon_BD_total_per_user_lag2_isna,log_coupon_BD_total_per_user_roll4,log_coupon_BD_total_per_user_roll4_isna,log_coupon_CDP_total_per_user_lag1,log_coupon_CDP_total_per_user_lag1_isna,log_coupon_CDP_total_per_user_lag2,log_coupon_CDP_total_per_user_lag2_isna,log_coupon_CDP_total_per_user_roll4,log_coupon_CDP_total_per_user_roll4_isna,log_coupon_folk_total_per_user_lag1,log_coupon_folk_total_per_user_lag1_isna,log_coupon_folk_total_per_user_lag2,log_coupon_folk_total_per_user_lag2_isna,log_coupon_folk_total_per_user_roll4,log_coupon_folk_total_per_user_roll4_isna,log_coupon_growth_other_total_per_user_lag1,log_coupon_growth_other_total_per_user_lag1_isna,log_coupon_growth_other_total_per_user_lag2,log_coupon_growth_other_total_per_user_lag2_isna,log_coupon_growth_other_total_per_user_roll4,log_coupon_growth_other_total_per_user_roll4_isna,log_coupon_MGM_total_per_user_lag1,log_coupon_MGM_total_per_user_lag1_isna,log_coupon_MGM_total_per_user_lag2,log_coupon_MGM_total_per_user_lag2_isna,log_coupon_MGM_total_per_user_roll4,log_coupon_MGM_total_per_user_roll4_isna,log_coupon_MKT_total_per_user_lag1,log_coupon_MKT_total_per_user_lag1_isna,log_coupon_MKT_total_per_user_lag2,log_coupon_MKT_total_per_user_lag2_isna,log_coupon_MKT_total_per_user_roll4,log_coupon_MKT_total_per_user_roll4_isna,log_coupon_register_total_per_user_lag1,log_coupon_register_total_per_user_lag1_isna,log_coupon_register_total_per_user_lag2,log_coupon_register_total_per_user_lag2_isna,log_coupon_register_total_per_user_roll4,log_coupon_register_total_per_user_roll4_isna,log_coupon_daily_total_per_user_lag1,log_coupon_daily_total_per_user_lag1_isna,log_coupon_daily_total_per_user_lag2,log_coupon_daily_total_per_user_lag2_isna,log_coupon_daily_total_per_user_roll4,log_coupon_daily_total_per_user_roll4_isna,log_coupon_total_per_user_lag1,log_coupon_total_per_user_lag1_isna,log_coupon_total_per_user_lag2,log_coupon_total_per_user_lag2_isna,log_coupon_total_per_user_roll4,log_coupon_total_per_user_roll4_isna,delta_trip_per_user,delta_nonrepeat_per_user
0,2025-11-03,15x2元1張,隨機組,14天在其他尖峰預估車資,中區,0.217617,0.274611,5.267858,0.000000,0.000000,0.0,0,0,0.000000,1,0.000

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2856 entries, 0 to 2855
Columns: 117 entries, experiment_date to delta_nonrepeat_per_user
dtypes: float64(59), int64(53), object(5)
memory usage: 2.5+ MB


In [41]:
df.describe()

,trip_cnt_per_user,nonrepeat_cnt_per_user,log1p_user_cnt,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day,has_national_holiday,trip_cnt_per_user_lag1,trip_cnt_per_user_lag1_isna,trip_cnt_per_user_lag2,trip_cnt_per_user_lag2_isna,trip_cnt_per_user_roll4,trip_cnt_per_user_roll4_isna,nonrepeat_cnt_per_user_lag1,nonrepeat_cnt_per_user_lag1_isna,nonrepeat_cnt_per_user_lag2,nonrepeat_cnt_per_user_lag2_isna,nonrepeat_cnt_per_user_roll4,nonrepeat_cnt_per_user_roll4_isna,weekday_nonrepeat_cnt_per_user_lag1,weekday_nonrepeat_cnt_per_user_lag1_isna,weekday_nonrepeat_cnt_per_user_lag2,weekday_nonrepeat_cnt_per_user_lag2_isna,weekday_nonrepeat_cnt_per_user_roll4,weekday_nonrepeat_cnt_per_user_roll4_isna,weekend_nonrepeat_cnt_per_user_lag1,weekend_nonrepeat_cnt_per_user_lag1_isna,weekend_nonrepeat_cnt_per_user_lag2,weekend_nonrepeat_cnt_per_user_lag2_isna,weekend_nonrepeat_cnt_per_user_roll4,weekend_nonrepeat_cnt_per_user_roll4_isna,weekday_trip_cnt_per_user_lag1,weekday_trip_cnt_per_user_lag1_isna,weekday_trip_cnt_per_user_lag2,weekday_trip_cnt_per_user_lag2_isna,weekday_trip_cnt_per_user_roll4,weekday_trip_cnt_per_user_roll4_isna,weekend_trip_cnt_per_user_lag1,weekend_trip_cnt_per_user_lag1_isna,weekend_trip_cnt_per_user_lag2,weekend_trip_cnt_per_user_lag2_isna,weekend_trip_cnt_per_user_roll4,weekend_trip_cnt_per_user_roll4_isna,weekday_match_rate_lag1,weekday_match_rate_lag1_isna,weekday_match_rate_lag2,weekday_match_rate_lag2_isna,weekday_match_rate_roll4,weekday_match_rate_roll4_isna,weekend_match_rate_lag1,weekend_match_rate_lag1_isna,weekend_match_rate_lag2,weekend_match_rate_lag2_isna,weekend_match_rate_roll4,weekend_match_rate_roll4_isna,log_coupon_BD_total_per_user_lag1,log_coupon_BD_total_per_user_lag1_isna,log_coupon_BD_total_per_user_lag2,log_coupon_BD_total_per_user_lag2_isna,log_coupon_BD_total_per_user_roll4,log_coupon_BD_total_per_user_roll4_isna,log_coupon_CDP_total_per_user_lag1,log_coupon_CDP_total_per_user_lag1_isna,log_coupon_CDP_total_per_user_lag2,log_coupon_CDP_total_per_user_lag2_isna,log_coupon_CDP_total_per_user_roll4,log_coupon_CDP_total_per_user_roll4_isna,log_coupon_folk_total_per_user_lag1,log_coupon_folk_total_per_user_lag1_isna,log_coupon_folk_total_per_user_lag2,log_coupon_folk_total_per_user_lag2_isna,log_coupon_folk_total_per_user_roll4,log_coupon_folk_total_per_user_roll4_isna,log_coupon_growth_other_total_per_user_lag1,log_coupon_growth_other_total_per_user_lag1_isna,log_coupon_growth_other_total_per_user_lag2,log_coupon_growth_other_total_per_user_lag2_isna,log_coupon_growth_other_total_per_user_roll4,log_coupon_growth_other_total_per_user_roll4_isna,log_coupon_MGM_total_per_user_lag1,log_coupon_MGM_total_per_user_lag1_isna,log_coupon_MGM_total_per_user_lag2,log_coupon_MGM_total_per_user_lag2_isna,log_coupon_MGM_total_per_user_roll4,log_coupon_MGM_total_per_user_roll4_isna,log_coupon_MKT_total_per_user_lag1,log_coupon_MKT_total_per_user_lag1_isna,log_coupon_MKT_total_per_user_lag2,log_coupon_MKT_total_per_user_lag2_isna,log_coupon_MKT_total_per_user_roll4,log_coupon_MKT_total_per_user_roll4_isna,log_coupon_register_total_per_user_lag1,log_coupon_register_total_per_user_lag1_isna,log_coupon_register_total_per_user_lag2,log_coupon_register_total_per_user_lag2_isna,log_coupon_register_total_per_user_roll4,log_coupon_register_total_per_user_roll4_isna,log_coupon_daily_total_per_user_lag1,log_coupon_daily_total_per_user_lag1_isna,log_coupon_daily_total_per_user_lag2,log_coupon_daily_total_per_user_lag2_isna,log_coupon_daily_total_per_user_roll4,log_coupon_daily_total_per_user_roll4_isna,log_coupon_total_per_user_lag1,log_coupon_total_per_user_lag1_isna,log_coupon_total_per_user_lag2,log_coupon_total_per_user_lag2_isna,log_coupon_total_per_user_roll4,log_coupon_total_per_user_roll4_isna,delta_trip_per_user,delta_nonrepeat_per_user
count,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000000,2856.000